# Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks

图像到图像的转换是一类视觉问题，其目标是学习训练数据中对齐的图像对之间输入到输出的映射。但是，在很多任务中并不存在配对的训练数据。我们提出一种在没有配对训练数据情况下，将输入数据从原域 X 映射到目标域 Y 的方法。我们的目标是学习映射G：X→Y，使得G（X）中的图像分布与使用对抗性损失的分布Y不可区分。因为这个映射是高度欠约束的，所以我们将它与一个逆映射F：Y→X耦合起来，引入一个循环一致性损失来强制 $ F(G(x)) \approx X $（反之亦然）。这种方法适用于风格转换，目标形变，季节转移，照片增强等）等任务。

## 摘要  

1 该文章提出了一种可以不用匹配成对的数据进行图像到图像进行转换的方法，缺乏对偶实例的监督情况下，在集合层面上进行监督学习  
2 “循环一致”的性质，就是说，如果把一个从英语到法语的句子翻译过来，然后从法语翻译成英语，应该回到原来的句子。在数学上，如果有一个翻译器G：X→Y和另一个翻译器F：Y→X，那么 G 和 F 应该是彼此逆的，并且这两个映射应该是双射。通过同时训练映射 G 和 F 来应用这个结构假设，并增加一个鼓励 $ F(G(x)) \approx X $ 和 $ G(F(x)) \approx Y $ 的循环一致性损失。将这种损失与 X 域和 Y 域的对抗性损失相结合，就可以完全实现对不成对的图像转换的目标。  
3 generator 用来将输入的两幅图像转换成对方的风格，discriminator用来检查生成图像的质量

## 1 数据准备  

1 输入数据是成对出现的，将待转换的两幅图同时送入GAN中  
2 dataA和dataB是两组需要进行图像转换的数据，不用一一对应

In [ ]:
np.random.shuffle(dataA)
np.random.shuffle(dataB)
batch_files = list(zip(dataA[idx * self.batch_size:(idx + 1) * self.batch_size],
                   dataB[idx * self.batch_size:(idx + 1) * self.batch_size]))
batch_images = [load_train_data(batch_file, args.load_size, args.fine_size) for batch_file in batch_files]
batch_images = np.array(batch_images).astype(np.float32)

## 2 损失函数g_loss  

1 real_A 和 real_B 是构造出来送入训练的图像对  
2 fake_B 和 fake_A 是分别根据real_A 和 real_B 由generator生成的图像  
3 fake_A_ 和 fake_B_是分别根据fake_B 和 fake_A 由generator生成的图像  
4 DB_fake 和 DA_fake 是分别将fake_B 和 fake_A送入discriminator输出的结果  
5 criterionGAN(self.DA_fake, tf.ones_like(self.DA_fake)) 和 criterionGAN(self.DB_fake, tf.ones_like(self.DB_fake)) 是欺骗discriminator的对抗性损失函数，让discriminator将生成的图像分类的真实的图像  
6 abs_criterion(self.real_A, self.fake_A_) 和 abs_criterion(self.real_B, self.fake_B_) 用来优化generator，使生成的图像接近真实的图像

In [ ]:
self.real_data = tf.placeholder(tf.float32,
                                [None, self.image_size, self.image_size,
                                 self.input_c_dim + self.output_c_dim],
                                name='real_A_and_B_images')

self.real_A = self.real_data[:, :, :, :self.input_c_dim]
self.real_B = self.real_data[:, :, :, self.input_c_dim:self.input_c_dim + self.output_c_dim]

self.fake_B = self.generator(self.real_A, self.options, False, name="generatorA2B")
self.fake_A_ = self.generator(self.fake_B, self.options, False, name="generatorB2A")
self.fake_A = self.generator(self.real_B, self.options, True, name="generatorB2A")
self.fake_B_ = self.generator(self.fake_A, self.options, True, name="generatorA2B")

self.DB_fake = self.discriminator(self.fake_B, self.options, reuse=False, name="discriminatorB")
self.DA_fake = self.discriminator(self.fake_A, self.options, reuse=False, name="discriminatorA")

self.g_loss = self.criterionGAN(self.DA_fake, tf.ones_like(self.DA_fake)) \
    + self.criterionGAN(self.DB_fake, tf.ones_like(self.DB_fake)) \
    + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
    + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)


criterionGAN = mae_criterion
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))

def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)       
    

## 3 损失函数d_loss  

1 fake_A_sample和fake_B_sample 分别是由real_B 和 real_A 生成的假的图像  
2 d_loss 是优化discriminator的损失函数，又来将real_B 和 real_A 与 fake_A_sample和fake_B_sample 区分出来

In [ ]:
self.fake_A_sample = tf.placeholder(tf.float32,
                                    [None, self.image_size, self.image_size,
                                     self.input_c_dim], name='fake_A_sample')
self.fake_B_sample = tf.placeholder(tf.float32,
                                    [None, self.image_size, self.image_size,
                                     self.output_c_dim], name='fake_B_sample')

self.DB_real = self.discriminator(self.real_B, self.options, reuse=True, name="discriminatorB")
self.DA_real = self.discriminator(self.real_A, self.options, reuse=True, name="discriminatorA")
self.DB_fake_sample = self.discriminator(self.fake_B_sample, self.options, reuse=True, name="discriminatorB")
self.DA_fake_sample = self.discriminator(self.fake_A_sample, self.options, reuse=True, name="discriminatorA")

self.db_loss_real = self.criterionGAN(self.DB_real, tf.ones_like(self.DB_real))
self.db_loss_fake = self.criterionGAN(self.DB_fake_sample, tf.zeros_like(self.DB_fake_sample))
self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2
self.da_loss_real = self.criterionGAN(self.DA_real, tf.ones_like(self.DA_real))
self.da_loss_fake = self.criterionGAN(self.DA_fake_sample, tf.zeros_like(self.DA_fake_sample))
self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2
self.d_loss = self.da_loss + self.db_loss

## 4 训练网络  

1 d_optim 和 g_optim 分别用来优化discriminator和generator  
2 现将输入图片送到generator，生成假的fake_A 和 fake_B，然后送入discriminator 进行训练

In [ ]:
self.d_optim = tf.train.AdamOptimizer(self.lr, beta1=args.beta1) \
    .minimize(self.d_loss, var_list=self.d_vars)
self.g_optim = tf.train.AdamOptimizer(self.lr, beta1=args.beta1) \
    .minimize(self.g_loss, var_list=self.g_vars)

# Update G network and record fake outputs
fake_A, fake_B, _, summary_str = self.sess.run(
    [self.fake_A, self.fake_B, self.g_optim, self.g_sum],
    feed_dict={self.real_data: batch_images, self.lr: lr})

# Update D network
_, summary_str = self.sess.run(
    [self.d_optim, self.d_sum],
    feed_dict={self.real_data: batch_images,
               self.fake_A_sample: fake_A,
               self.fake_B_sample: fake_B,
               self.lr: lr})